# Radial Velocity Curves

<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#The-2-body-problem" data-toc-modified-id="The-2-body-problem-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>The 2-body problem</a></span><ul class="toc-item"><li><span><a href="#Coordinates" data-toc-modified-id="Coordinates-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Coordinates</a></span></li><li><span><a href="#Stepping" data-toc-modified-id="Stepping-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Stepping</a></span></li></ul></li><li><span><a href="#Defining-the-simulation" data-toc-modified-id="Defining-the-simulation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Defining the simulation</a></span></li><li><span><a href="#Interactive-plotting" data-toc-modified-id="Interactive-plotting-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Interactive plotting</a></span></li></ul></div>

In [7]:
%matplotlib inline

import time

import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 16

from ipywidgets import interact, Layout
import ipywidgets as w

from astropy import units as u
from astropy.constants import G

## The 2-body problem

For a single-planet system, the star and planet are in elliptical orbits sharing a common focus at the center of mass. At any time, the line joining the two bodies must pass through this common focus.

Our calculations will assume that a system of two bodies in mutual orbit can be treated as mathematically equivalent to an object with reduced mass $\mu$ in orbit around a _stationary_ mass $M$ corresponding to the combined center of mass of the bodies. This gives us a 1-body problem which is relatively easy to model. Once we have the radius $r$ and angle $\theta$ for a point in the CoM reference frame, it is simple arithmetic to get the postions of each body.

From Kepler's Third Law, the semi-major axis of the binary system is 
$$a = \sqrt[3]{\frac{P^2 G (m_1+m_2)}{4 \pi^2}}$$

The reduced mass is: $$\mu = \frac{m_1 m_2}{m_1 + m_2}$$

The individual bodies have semi-major axes about the center of mass: $$a_1 = \frac{\mu}{m_1} a \qquad a_2 = \frac{\mu}{m_2} a$$
For convenience, we define $M = m_1+m_2$.

### Coordinates
We are ignoring inclination of the orbital plane, as this merely scales the velocities without changing anything interesting in the simulation. This reduces it to a 2-D problem with the orbit in a plane that includes our line of sight.

We need to take two angles into account: the planet's position on the orbit, $\theta$, and the angle between pericenter and our line of sight, $\varpi$ (called varpi in the code and plots).

The sim always starts at pericenter, taken as $\theta=0$

### Stepping
In the simulation, we need to calculate the angular step $d\theta$ for each time step $dt$. This is Kepler II, a manifestation of angular momentum conservation.

The angular momentum is
$$ L = \mu \sqrt{G M a (1-e^2)} $$
and 
$$ \frac{dA}{dt} = \frac{1}{2} r^2 \frac{d \theta}{dt} = \frac{1}{2} \frac{L}{\mu} \quad \Rightarrow \quad d \theta = \frac{dA}{dt} \frac{2}{r^2} dt $$

By Kepler II, $\frac{dA}{dt}$ is constant and $d \theta \sim r^{-2}$: varying around the orbit for non-zero eccentricities.

## Defining the simulation

In [14]:
def runSim(m_p, P, e, varpi, m_star):
    "Main orbit loop. Parameters should have appropriate units."

    # initialize parameters
    M = m_star + m_p
    a = ((P**2 * G * M)/(4*np.pi**2))**(1/3)
    mu = (m_star * m_p)/M

    N = 500 # steps around the orbit
    t = 0 * u.s
    dt = P/N # time step
    theta = 0 * u.rad
    
    # calculate angular momentum about center of mass
    L_ang = mu * np.sqrt(G * M * a * (1 - e**2))
    
    # dA/dt, for calculating dtheta at each time step
    dAdt = L_ang / (2 * mu)
    
    # initialize output arrays,    
    t_P = np.zeros(N) # t/P, fraction of the orbit [dimensionless]
    v_r_star = np.zeros(N) * u.m/u.s # radial velocity, star
    
    # run the time-step loop
    for step in range(N):
        # Calculate orbit parameters in the CoM reference frame,
        # then translate to individual body positions
        
        # position
        r =  a*(1 - e**2)/(1 + e*np.cos(theta))
        
        # velocity
        v = np.sqrt(G * M * (2/r - 1/a))
        
        # radial velocity for mu (along our line of sight)
        vr  = -v * np.sin(theta + varpi)
        # radial velocity for star
        v1r = mu/m_star * vr
        
        # store the results
        t_P[step] = t/P
        v_r_star[step] = v1r
        
        # prepare for next step
        dtheta = 2 * dAdt/r**2 * dt * u.rad
        theta += dtheta
        t += dt

    return t_P, v_r_star

The sim only runs for a single orbit, but the results are duplicated for plotting 2 orbits. Scientifically meaningless but visually helpful.

In [15]:
def plotRV(m_p, m_p_unit, P, P_unit, e, varpi, m_star):
    # add units
    if m_p_unit=='M_earth':
        m_p *= u.M_earth
    else:
        m_p *= u.M_jup
    m_star *= u.M_sun
    if P_unit=='day':
        P *= u.day
    else:
        P *= u.year
    varpi *= u.deg
    
    # run the simulation
    t_P, v_r_star = runSim(m_p, P, e, varpi, m_star)

    # plotting
    x = np.hstack( (t_P, 1+t_P) )
    y = np.hstack( (v_r_star, v_r_star) )

    plt.figure(figsize=(15, 5))
    plt.plot(x, y)
    plt.xlabel('t/P')
    plt.ylabel('RV (m/s)')
    plt.title(f"Planet: {m_p}, star: {m_star}, P: {P}, e: {e}, varpi: {varpi}");

## Interactive plotting

Ugly layout at this stage - it would be good to have units dropdowns alongside the associated slider, but that's not very easy with widgets. On the TODO list...

In [13]:
style = {'description_width': 'initial'} # to avoid the labels getting truncated
interact(plotRV, 
    m_star = w.FloatSlider(description="star mass ($M_{\odot}$)", style=style,
                            layout=Layout(width='80%'),
                            continuous_update=False, 
                            min=0.1, max=10.0, value=1),
    m_p = w.FloatSlider(description="planet mass", style=style,
                            layout=Layout(width='80%'),
                            continuous_update=False, 
                            min=0.1, max=10.0, value=1),
    m_p_unit = w.RadioButtons(description="Planet unit",
                            options=['M_Earth', 'M_Jup']),
    P = w.FloatSlider(description="Orbit period", style=style,
                            layout=Layout(width='80%'),
                            continuous_update=False, 
                            min=1.0, max=50.0, step=0.1, value=20),
    P_unit = w.RadioButtons(description="Period unit",
                            options=['day', 'year']),
    e = w.FloatSlider(description="eccentricity", style=style,
                            layout=Layout(width='80%'),
                            continuous_update=False, 
                            min=0.0, max=0.9, step=0.01, value=0),
    varpi = w.FloatSlider(description="varpi (deg)", style=style,
                            layout=Layout(width='80%'),
                            continuous_update=False,
                            min=-90.0, max=90.0, value=0) );

interactive(children=(FloatSlider(value=1.0, continuous_update=False, description='planet mass', layout=Layout…